In [1]:
from datetime import date
import pandas as pd
import numpy as np
from math import sqrt
from src.models.liquidity_distribution import Pool, LiquidityPeriphery 
import warnings
warnings.filterwarnings('ignore')
from src.data_processing.fetch_data import fetch_data_subgraph
pd.set_option("display.max_columns",100)
%load_ext autoreload
%autoreload 2
from src.data_processing.fetch_data import fetch_oku_liquidity
POOLS = [
    (3, "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", "v3 USDC/ETH 0.3%", ),
    (3, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "v3 USDC/ETH 0.05%", ),
]
API_KEY = "893d7471304c5edf436c8ba60781762c"
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640".lower()

In [2]:
data = fetch_oku_liquidity(pool_address=POOL_ADDRESS, block_number=20719648)

In [3]:
query = """
query ($poolAddress: String!, $skip: Int!) {
  poolDayDatas(
    where: {pool: $poolAddress}
    first: 1000
    skip: $skip
    orderBy: date
    orderDirection: desc
  ) {
    date
    feesUSD
    liquidity
    sqrtPrice
    token0Price
    token1Price
    tvlUSD
    volumeToken0
    volumeToken1
    volumeUSD
    txCount
  }
}
"""

# Variables for the query
variables = {
    "poolAddress": POOL_ADDRESS,
    "skip": 0
}

# Fetch the data
pool_data = fetch_data_subgraph(
    API_KEY,
    query,
    variables=variables,
    data_key="poolDayDatas",
    first_n=None,  # Set this to a number if you want to limit the results
    batch_size=1000
)

Fetched 1000 records
Fetched 1241 records


In [4]:
df = pd.DataFrame(pool_data)

In [5]:
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    totalValueLockedUSD
    totalValueLockedETH
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
# POOL_ADDRESS = "0xc7bBeC68d12a0d1830360F8Ec58fA599bA1b0e9b".lower()
variables = {'pool_id': POOL_ADDRESS}
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]

l = pd.read_parquet("/Users/gnapsamuel/Documents/AMM/proteus-LP-backtesting/data/uniswap/USDC-WETH/fee_500/liquidity/liquidity_distribution_20773531.parquet")

pool = Pool(pool_address=POOL_ADDRESS,
            token0=data_pool['token0']['symbol'],
            token1=data_pool['token1']['symbol'],
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])


Fetched 1 records


In [10]:
# Example usage
print(pool.token0, pool.token1)
amount0_raw = 2328*2
amount1_raw = 1*2
current_price = 2328
lower_price = 2217
upper_price = 2450


lower_tick = 198280
upper_tick = 199280
current_tick = 198780

calculator = LiquidityPeriphery(pool)
sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, amount0, amount1, liquidity = calculator.calculate_liquidity(amount0_raw, amount1_raw, lower_tick, upper_tick, current_tick)


USDC WETH


In [17]:
liquidity

3906149178000611

In [11]:
sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, amount0, amount1, liquidity

(1641030417809999877662689105714598,
 1600515232939513656535723291125537,
 1682571197546006325973824888772292,
 4656000000,
 2000000000000000000,
 3906149178000611)

In [12]:
amount0, amount1 = calculator.get_amounts_for_liquidity(sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, liquidity)

In [13]:
amount0, amount1

(4656.0, 1.9975)

In [15]:
(3906149178000611/7.881445837741723e+20)*100

0.0004956132743176781